In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [12]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [13]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


Let's run this computation and iterate:

In [7]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 19.801123
Training accuracy: 9.7%
Validation accuracy: 11.5%
Loss at step 100: 2.237672
Training accuracy: 71.4%
Validation accuracy: 70.8%
Loss at step 200: 1.809805
Training accuracy: 75.0%
Validation accuracy: 73.1%
Loss at step 300: 1.579807
Training accuracy: 76.2%
Validation accuracy: 73.9%
Loss at step 400: 1.423616
Training accuracy: 77.1%
Validation accuracy: 74.2%
Loss at step 500: 1.307445
Training accuracy: 77.7%
Validation accuracy: 74.7%
Loss at step 600: 1.216172
Training accuracy: 78.4%
Validation accuracy: 74.7%
Loss at step 700: 1.141620
Training accuracy: 78.9%
Validation accuracy: 74.9%
Loss at step 800: 1.078963
Training accuracy: 79.3%
Validation accuracy: 75.0%
Test accuracy: 82.4%


# practise!

In [18]:
# train a logistic classifier for train_dataset and train_labels
# define graph
graph = tf.Graph()
with graph.as_default():
    # define dataset
    tf_train_dataset = tf.constant(train_dataset[:10000,:])
    tf_train_labels = tf.constant(train_labels[:10000])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    # define variables (weights, biases) with initial values
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # define loss
    logits = tf.matmul(tf_train_dataset,weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
    
    # define optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # define predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset,weights)+biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+biases)
    
    
def accuracy(prediction,truth):
    return np.sum(np.argmax(prediction,axis=1)==np.argmax(truth,axis=1))/prediction.shape[0]

with tf.Session(graph=graph) as session:
    # initialize
    tf.initialize_all_variables().run()
    # train
    for step in range(801):
        _,l,pred = session.run([optimizer,loss,train_prediction])
        if step % 100 ==0:
            print('step: {}, loss: {}, accuracy:{}, valid accuracy:{}'.format(step,l,accuracy(pred,train_labels[:10000]),accuracy(valid_prediction.eval(),valid_labels)))
    print('Training done!')
    print('test accuracy: ', accuracy(test_prediction.eval(), test_labels))

step: 0, loss: 18.48882293701172, accuracy:0.0921, valid accuracy:0.1185
step: 100, loss: 2.2325899600982666, accuracy:0.7147, valid accuracy:0.6979
step: 200, loss: 1.782598614692688, accuracy:0.7483, valid accuracy:0.7259
step: 300, loss: 1.549940586090088, accuracy:0.7625, valid accuracy:0.7358
step: 400, loss: 1.3974037170410156, accuracy:0.7725, valid accuracy:0.7397
step: 500, loss: 1.2853747606277466, accuracy:0.779, valid accuracy:0.742
step: 600, loss: 1.1978954076766968, accuracy:0.7852, valid accuracy:0.7454
step: 700, loss: 1.126826524734497, accuracy:0.7896, valid accuracy:0.7465
step: 800, loss: 1.0674121379852295, accuracy:0.7936, valid accuracy:0.7471
Training done!
test accuracy:  0.8241


Let's now switch to stochastic gradient descent training instead, which is much faster.
The graph will be similar, except that instead of holding all the training data into a constant node, we create a Placeholder node which will be fed actual data at every call of session.run().

In [20]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

let's run it

In [21]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.391232
Minibatch accuracy: 10.9%
Validation accuracy: 10.7%
Minibatch loss at step 500: 1.293702
Minibatch accuracy: 75.8%
Validation accuracy: 75.5%
Minibatch loss at step 1000: 1.453120
Minibatch accuracy: 74.2%
Validation accuracy: 76.6%
Minibatch loss at step 1500: 0.900042
Minibatch accuracy: 83.6%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 0.813327
Minibatch accuracy: 86.7%
Validation accuracy: 77.3%
Minibatch loss at step 2500: 0.998739
Minibatch accuracy: 77.3%
Validation accuracy: 78.3%
Minibatch loss at step 3000: 1.012559
Minibatch accuracy: 77.3%
Validation accuracy: 78.9%
Test accuracy: 86.1%


# practise! 

In [31]:
# train a logistic classifier for train_dataset and train_labels
# define graph
batch_size = 128
graph = tf.Graph()
with graph.as_default():
    # define dataset
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    # define variables (weights, biases) with initial values
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # define loss
    logits = tf.matmul(tf_train_dataset,weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
    
    # define optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # define predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset,weights)+biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,weights)+biases)
    
    
def accuracy(prediction,truth):
    return np.sum(np.argmax(prediction,axis=1)==np.argmax(truth,axis=1))/prediction.shape[0]

with tf.Session(graph=graph) as session:
    # initialize
    tf.initialize_all_variables().run()
    # train
    for step in range(801):
        # stochastic
        offset = np.random.randint(1,train_dataset.shape[0]-batch_size)
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        _,l,pred = session.run([optimizer,loss,train_prediction],feed_dict = feed_dict)
        if step % 100 ==0:
            print('step: {}, loss: {}, accuracy:{}, valid accuracy:{}'.format(step,l,accuracy(pred,batch_labels),accuracy(valid_prediction.eval(),valid_labels)))
    print('Training done!')
    print('test accuracy: ', accuracy(test_prediction.eval(), test_labels))

/Users/harryxu/anaconda/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py:32: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


step: 0, loss: 17.25391960144043, accuracy:0.0, valid accuracy:0.0972
step: 100, loss: 1.958814263343811, accuracy:0.0, valid accuracy:0.7159
step: 200, loss: 2.3009536266326904, accuracy:0.0, valid accuracy:0.7382
step: 300, loss: 1.9007123708724976, accuracy:0.0, valid accuracy:0.7478
step: 400, loss: 1.9395300149917603, accuracy:0.0, valid accuracy:0.7559
step: 500, loss: 1.5003092288970947, accuracy:0.0, valid accuracy:0.7575
step: 600, loss: 1.489224910736084, accuracy:0.0, valid accuracy:0.7633
step: 700, loss: 1.1347218751907349, accuracy:0.0, valid accuracy:0.7643
step: 800, loss: 1.6497299671173096, accuracy:0.0, valid accuracy:0.7581
Training done!
test accuracy:  0.831


In [29]:
np.random.randint(1,10)

7

# Problem
Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.

In [24]:
graph = tf.Graph()
fc1_size = 1024
with graph.as_default():
    # data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # variables
    fc1_weights = tf.Variable(tf.truncated_normal([image_size*image_size, fc1_size]))
    fc1_biases = tf.Variable(tf.zeros([fc1_size]))
    
    
    output_weights = tf.Variable(tf.truncated_normal([fc1_size,num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))
    
    # score, loss
    fc1 = tf.matmul(tf_train_dataset,fc1_weights)+fc1_biases
    logits = tf.matmul(tf.nn.relu(fc1),output_weights) + output_biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset,fc1_weights)+fc1_biases), output_weights)+output_biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,fc1_weights)+fc1_biases), output_weights)+output_biases)
    
# tf session
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    # run steps
    for step in range(2001):
        offset = np.random.randint(1,train_dataset.shape[0]-batch_size)
        batch_train = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        feed_dict = {tf_train_dataset:batch_train, tf_train_labels:batch_labels}
        _,l,pred = session.run([optimizer,loss,train_prediction], feed_dict=feed_dict)
        if step%100 ==0:
            print('step: {}, train_loss: {}, train_accuracy: {}, valid_accuracy: {}'.format(step,l,accuracy(pred,batch_labels),
                  accuracy(valid_prediction.eval(),valid_labels)))
    print('Training Done!')
    print('test_accuracy:', accuracy(test_prediction.eval(),test_labels))
    

step: 0, train_loss: 315.92193603515625, train_accuracy: 0.0625, valid_accuracy: 0.4291
step: 100, train_loss: 26.270484924316406, train_accuracy: 0.796875, valid_accuracy: 0.7869
step: 200, train_loss: 16.406665802001953, train_accuracy: 0.8359375, valid_accuracy: 0.7935
step: 300, train_loss: 16.574703216552734, train_accuracy: 0.8203125, valid_accuracy: 0.802
step: 400, train_loss: 10.89886474609375, train_accuracy: 0.84375, valid_accuracy: 0.7717
step: 500, train_loss: 22.680160522460938, train_accuracy: 0.7578125, valid_accuracy: 0.8023
step: 600, train_loss: 46.15645217895508, train_accuracy: 0.6875, valid_accuracy: 0.7693
step: 700, train_loss: 10.568199157714844, train_accuracy: 0.7734375, valid_accuracy: 0.792
step: 800, train_loss: 2.832437753677368, train_accuracy: 0.8828125, valid_accuracy: 0.8106
step: 900, train_loss: 10.580953598022461, train_accuracy: 0.7578125, valid_accuracy: 0.8024
step: 1000, train_loss: 3.7352592945098877, train_accuracy: 0.8359375, valid_accuracy:

In [28]:
# more organized version:
def accuracy(prediction,truth):
    return np.sum(np.argmax(prediction,axis=1)==np.argmax(truth,axis=1))/prediction.shape[0]

graph = tf.Graph()
h1_wsize = 1024
batch_size = 128

with graph.as_default():
    # data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # weights
    weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size*image_size, h1_wsize])),
        'out': tf.Variable(tf.truncated_normal([h1_wsize, num_labels]))
    }
    biases = {
        'h1': tf.Variable(tf.zeros([h1_wsize])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }
    # 1 layer perceptron model
    def mlp(x,weights,biases):
        layer_1 = tf.matmul(x, weights['h1']) + biases['h1']
        layer_1 = tf.nn.relu(layer_1)
        
        out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
        return out_layer
    
    logits = mlp(tf_train_dataset, weights, biases)      
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = mlp(tf_valid_dataset,weights,biases)
    test_prediction = mlp(tf_test_dataset,weights,biases)
    
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for step in range(801):
        offset = np.random.randint(1,train_dataset.shape[0]-batch_size)
        batch_train = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        feed_dict = {tf_train_dataset:batch_train, tf_train_labels:batch_labels}
        _,l,pred = session.run([optimizer,loss,train_prediction], feed_dict=feed_dict)
        if step%100 ==0:
            print('step: {}, train_loss: {}, train_accuracy: {}, valid_accuracy: {}'.format(step,l,accuracy(pred,batch_labels),
                  accuracy(valid_prediction.eval(),valid_labels)))
    print('Training Done!')
    print('test_accuracy:', accuracy(test_prediction.eval(),test_labels))
    
    


step: 0, train_loss: 436.2685241699219, train_accuracy: 0.109375, valid_accuracy: 0.422
step: 100, train_loss: 31.951557159423828, train_accuracy: 0.765625, valid_accuracy: 0.7647
step: 200, train_loss: 38.33210372924805, train_accuracy: 0.7109375, valid_accuracy: 0.7778
step: 300, train_loss: 23.820541381835938, train_accuracy: 0.7734375, valid_accuracy: 0.7818
step: 400, train_loss: 9.822334289550781, train_accuracy: 0.84375, valid_accuracy: 0.8034
step: 500, train_loss: 13.722471237182617, train_accuracy: 0.8125, valid_accuracy: 0.8044
step: 600, train_loss: 6.4842729568481445, train_accuracy: 0.859375, valid_accuracy: 0.8081
step: 700, train_loss: 13.399843215942383, train_accuracy: 0.8125, valid_accuracy: 0.8072
step: 800, train_loss: 20.254350662231445, train_accuracy: 0.765625, valid_accuracy: 0.7623
Training Done!
test_accuracy: 0.8252
